# Fitting a Llama7b Model with Lora

In [ ]:
import logging
import os
import huggingface_hub

import chai_guanaco as chai

from chaiverse.dataset import DatasetLoader, CausalDatasetBuilder
from chaiverse.tokenizer import LlamaTokenizer
from chaiverse.model.lora_model import LoraTrainer

## Login Setup

In [ ]:
chai.developer_login()

In [ ]:
huggingface_hub.login()

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
logging.basicConfig(level=logging.INFO)

## Load and process a small dataset

We load a small subset of Chai's conversational dataset

In [ ]:
# load data
data_path = 'ChaiML/davinci_completions_1.7m_unsampled'
data_loader = DatasetLoader(
        hf_path=data_path,
        data_samples=100,
        validation_split_size=0.1,
        shuffle=True,
        )
df = data_loader.load()
print(df)

In [ ]:
# A sample of the data. It includes `input_text` as prompt and `output_text` as response
df['train'][0]

In [ ]:
# process data
tokenizer = LlamaTokenizer()
data_builder = CausalDatasetBuilder(
        tokenizer_loader=tokenizer,
        block_size=1024,
        )
data = data_builder.generate(df, n_jobs=10)
print(data)

## Model setup and fitting

In [ ]:
# model setup
base_model = 'NousResearch/Llama-2-7b-hf'
model = LoraTrainer(
        model_name=base_model,
        output_dir='test_lora',
        )
print(model.lora_config)

In [ ]:
# model fitting
model.fit()

In [ ]:
# save model, default to model.output_dir
# can also specify save path : model.save(path=/alt/save/path)
model.save()

In [ ]:
# merge lora weights with base model weights
# Defult to use the lora weights saved in mode.output_dir
# can specify path to lora weights : model.merge(path=/alt/save/path)
model.merge()

In [ ]:
# push to huggingface
model_path = "your/huggingface/model/path"
model.push_to_hub(model_path, private=False)